In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from variables_config import (
    param_info_file, 
    plots_dir, 
    redshift_values, 
    lf_data_dir,
    colour_data_dir, 
    uvlf_limits, 
    colour_limits,
    filters
)

def read_colour(filename):
    """Read data from file."""
    try:
        data = pd.read_csv(filename, delim_whitespace=True)
        return data
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

def calculate_parameter_values(param_info):
    """
    Calculate all parameter values based on LogFlag.
    Returns dictionary with values for each variation (n2, n1, 0, 1, 2).
    """
    min_val = param_info['MinVal']
    max_val = param_info['MaxVal']
    fid_val = param_info['FiducialVal']
    
    if param_info['LogFlag']:
        # Logarithmic spacing
        log_min = np.log10(min_val)
        log_max = np.log10(max_val)
        log_fid = np.log10(fid_val)
        
        # Calculate n1 and 1 values in log space
        n1_val = 10**(log_min + (log_fid - log_min)/2)
        val_1 = 10**(log_fid + (log_max - log_fid)/2)
    else:
        # Linear spacing
        n1_val = min_val + (fid_val - min_val)/2
        val_1 = fid_val + (max_val - fid_val)/2
    
    return {
        'n2': min_val,    # MinVal
        'n1': n1_val,     # Calculated intermediate value
        '0': fid_val,     # FiducialVal
        '1': val_1,       # Calculated intermediate value
        '2': max_val      # MaxVal
    }

def get_safe_name(name, filter_system_only=False):
    """
    Convert string to path-safe version and/or extract filter system.
    
    Args:
        name: String to process (e.g., "GALEX FUV" or "UV1500")
        filter_system_only: If True, returns only the filter system (e.g., "GALEX" or "UV")
    
    Returns:
        Processed string (e.g., "GALEX_FUV" or "GALEX")
    """
    # Replace spaces with underscores
    safe_name = name.replace(' ', '_')
    
    # If we only want the filter system, return the first part
    if filter_system_only:
        return safe_name.split('_')[0]
    
    return safe_name

def get_colour_dir_name(band1, band2):
    """
    Create a standardized directory name for colour plots.
    Examples:
        ("GALEX FUV", "GALEX NUV") -> "GALEX_FUV-NUV"
        ("UVM2", "SUSS") -> "UVM2-SUSS"
    """
    # Extract the relevant parts of the filter names
    if ' ' in band1:
        system1, filter1 = band1.split(' ', 1)
    else:
        system1, filter1 = band1, band1

    if ' ' in band2:
        system2, filter2 = band2.split(' ', 1)
    else:
        system2, filter2 = band2, band2
        
    # If both filters are from the same system, use shortened version
    if system1 == system2:
        return f"{get_safe_name(system1)}_{filter1}-{filter2}"
    else:
        return f"{get_safe_name(band1)}-{get_safe_name(band2)}"

In [ ]:
def plot_parameter_variations_uvlf(param_num, param_info, redshift, band_type, band):
    """Plot UVLF showing all parameter variations at one redshift."""
    # This is your existing plot_all_uvlf_variations function
    param_values = calculate_parameter_values(param_info)
    
    plt.figure(figsize=(10, 8))
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    # Get filter system from band name
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    # Plot each variation
    for var, colour in zip(variations, colours):
        filename = os.path.join(get_safe_name(base_output_dir), redshift['label'],
                              f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
        
        if os.path.exists(filename):
            data = pd.read_csv(filename, delimiter='\t')
            label = f"{param_info['ParamName']} = {param_values[var]:.3g}"
            plt.errorbar(data['magnitude'], data['phi'], 
                        yerr=data['phi_sigma'],
                        fmt='o-', color=colour, 
                        label=label,
                        markersize=4, capsize=2)
        else:
            print(f"File not found: {filename}")

    if plt.gca().has_data():
        plt.xlabel('Magnitude (AB)', fontsize=12)
        plt.ylabel('Number Density (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        plt.xlim(*uvlf_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.6)
        plt.title(f'{param_info["Description"]} - {param_info["ParamName"]} (p{param_num})\n'
                 f'z = {redshift["redshift"]}',
                 fontsize=14)
        plt.legend(loc='upper left', title='Parameter Values')
        plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
        
        # Updated path to match new structure
        output_path = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 
                                 'parameter_variations',
                                 f'UVLF_p{param_num}_{param_info["ParamName"]}_z{redshift["label"]}.pdf')
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        print(f"Saving UVLF plot: {output_path}")
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
    
    plt.close()


# Then in your colour plotting functions, use it like this:
def plot_parameter_variations_colour(param_num, param_info, redshift, band_type, colour_pairs):
    """Plot colour distribution showing all parameter variations at one redshift."""
    param_values = calculate_parameter_values(param_info)
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    for band1, band2 in colour_pairs:
        plt.figure(figsize=(10, 8))
        
        # Use new function for directory name
        colour_dir = get_colour_dir_name(band1, band2)
        
        for var, colour in zip(variations, colours):
            # Create consistent filename
            colour_name = f"{band1.split()[-1]}-{band2.split()[-1]}"  # Keep original naming for files
            
            filename = os.path.join(get_safe_name(colour_data_dir[band_type]["GALEX"]), 
                                  redshift['label'],
                                  f"Colour_1P_p{param_num}_{var}_{colour_name}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                data = pd.read_csv(filename, delimiter='\t')
                plt.plot(data['colour'], data['distribution'], 
                        color=colour, linewidth=2,
                        label=f'{param_info["ParamName"]} = {param_values[var]:.3g}')
        
        plt.xlabel(f'{band1} - {band2} [mag]', fontsize=14)
        plt.ylabel('Normalized Count', fontsize=14)
        plt.xlim(*colour_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.3)
        plt.title(f'{param_info["Description"]} - {param_info["ParamName"]} (p{param_num})\n'
                 f'z = {redshift["redshift"]}', 
                 fontsize=16)
        plt.legend(title='Parameter Values', 
                  title_fontsize=12,
                  fontsize=11,
                  loc='upper right',
                  framealpha=0.95)
        
        plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
        plt.tight_layout()
        
        # Updated output path using new directory name
        output_dir = os.path.join(plots_dir, 'colours', band_type, colour_dir, 
                                'parameter_variations')
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, 
                                  f'Colour_p{param_num}_{param_info["ParamName"]}_z{redshift["label"]}.pdf')
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()


In [ ]:

def process_all_parameters():
    """Process all parameter variation plots."""
    colour_pairs = [
        ("GALEX FUV", "GALEX NUV"),
    ]
    
    params_df = pd.read_csv(param_info_file)
    
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        for snap, redshift in redshift_values.items():
            print(f"Processing redshift {redshift['redshift']}")
            
            for band_type in ['intrinsic', 'attenuated']:
                # Process UVLFs
                for band in filters[band_type]:
                    print(f"Plotting UVLF variations for {band_type} {band}")
                    plot_parameter_variations_uvlf(param_num, param_info, redshift, band_type, band)
                
                # Process colours
                print(f"Processing colour variations for z={redshift['redshift']}")
                plot_parameter_variations_colour(param_num, param_info, redshift, band_type, colour_pairs)

In [ ]:
def create_redshift_evolution_plot(param_num, param_info, band_type, band):
    """Create plot showing redshift evolution for each variation of a parameter."""
    param_values = calculate_parameter_values(param_info)
    
    variations = ['n2', 'n1', '1', '2']
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Redshift Evolution - {param_info["ParamName"]} (p{param_num})\n{param_info["Description"]}', 
                 fontsize=16, y=1.02)
    
    redshift_colours = {
        '044': 'darkred',     # z=2.00
        '052': 'orangered',   # z=1.48
        '060': 'orange',      # z=1.05
        '086': 'gold'         # z=0.10
    }
    
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    for ax, variation in zip(axes.flatten(), variations):
        for snap, redshift in redshift_values.items():
            filename = os.path.join(get_safe_name(base_output_dir), 
                                  redshift['label'],
                                  f"UVLF_1P_p{param_num}_{variation}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    
                    ax.errorbar(data['magnitude'], data['phi'], 
                              yerr=data['phi_sigma'],
                              fmt='o-', color=redshift_colours[snap],
                              label=f'z = {redshift["redshift"]}',
                              markersize=4, capsize=2)
                except Exception as e:
                    print(f"Error plotting {filename}: {e}")
        
        ax.set_xlim(-25, -16)
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.set_title(f'{param_info["ParamName"]} = {param_values[variation]:.3g}', 
                    fontsize=12)
        ax.set_xlabel('Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Φ (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.legend(fontsize=10, title='Redshift', title_fontsize=10)
    
    plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
    plt.tight_layout()
    
    output_dir = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 'redshift_variations')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 
                              f'UVLF_redshift_evolution_p{param_num}_{get_safe_name(param_info["ParamName"])}.pdf')
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

def create_parameter_variation_plot(param_num, param_info, band_type, band):
    """Create plot showing parameter variations at each redshift."""
    param_values = calculate_parameter_values(param_info)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Parameter Variations - {param_info["ParamName"]} (p{param_num})\n{param_info["Description"]}', 
                 fontsize=16, y=1.02)
    
    variation_colours = {
        'n2': 'blue',
        'n1': 'green',
        '1': 'red',
        '2': 'purple'
    }
    
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    for ax, (snap, redshift) in zip(axes.flatten(), redshift_values.items()):
        for var in ['n2', 'n1', '1', '2']:
            filename = os.path.join(get_safe_name(base_output_dir), 
                                  redshift['label'],
                                  f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    label = f'{param_info["ParamName"]} = {param_values[var]:.3g}'
                    
                    ax.errorbar(data['magnitude'], data['phi'], 
                              yerr=data['phi_sigma'],
                              fmt='o-', color=variation_colours[var],
                              label=label,
                              markersize=4, capsize=2)
                except Exception as e:
                    print(f"Error plotting {filename}: {e}")
        
        ax.set_xlim(-25, -16)
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.set_title(f'z = {redshift["redshift"]}', fontsize=12)
        ax.set_xlabel('Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Φ (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.legend(fontsize=9, title='Parameter Values', title_fontsize=10)
    
    plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
    plt.tight_layout()
    
    output_dir = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 'parameter_variations')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 
                              f'UVLF_parameter_variation_p{param_num}_{get_safe_name(param_info["ParamName"])}.pdf')
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
# process redshift and parameter variation plots. (4 plots per fig)
def process_all_variations():
    """Process both types of variation plots for all parameters and bands."""
    params_df = pd.read_csv(param_info_file)
    
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        for band_type in ['intrinsic', 'attenuated']:
            for band in filters[band_type]:
                print(f"Creating variation plots for {band_type} {band}")
                create_redshift_evolution_plot(param_num, param_info, band_type, band)
                create_parameter_variation_plot(param_num, param_info, band_type, band)

if __name__ == "__main__":
    process_all_variations()

In [ ]:

def create_parameter_grid(params_df, redshift, band_type, band_or_colour_pairs, plot_type='uvlf'):
    """Create grid plot of all parameters for a given redshift."""
    nrows, ncols = 7, 4  # 28 parameters in a 7x4 grid
    fig, axes = plt.subplots(nrows, ncols, figsize=(20, 35))
    
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    for param_num in range(1, 29):
        row = (param_num - 1) // ncols
        col = (param_num - 1) % ncols
        ax = axes[row, col]
        
        param_info = params_df.iloc[param_num-1]
        param_values = calculate_parameter_values(param_info)
        
        if plot_type == 'uvlf':
            filter_system = get_safe_name(band_or_colour_pairs, filter_system_only=True)
            base_dir = lf_data_dir[band_type][filter_system]
            
            for var, colour in zip(variations, colours):
                filename = os.path.join(get_safe_name(base_dir), redshift['label'],
                                    f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band_or_colour_pairs)}_{redshift['label']}_{band_type}.txt")
                
                if os.path.exists(filename):
                    data = pd.read_csv(filename, delimiter='\t')
                    ax.plot(data['magnitude'], data['phi'], color=colour, linewidth=1.5,
                           label=f'{param_values[var]:.3g}')
        
        else:  # colour plots
            band1, band2 = band_or_colour_pairs[0]  # Assuming single colour pair
            filter_system = get_safe_name(band1, filter_system_only=True)
            base_dir = colour_data_dir[band_type][filter_system]
            colour_name = f"{get_safe_name(band1).split('_')[-1]}-{get_safe_name(band2).split('_')[-1]}"
            
            for var, colour in zip(variations, colours):
                filename = os.path.join(get_safe_name(base_dir), redshift['label'],
                                    f"Colour_1P_p{param_num}_{var}_{colour_name}_{redshift['label']}_{band_type}.txt")
                
                if os.path.exists(filename):
                    data = pd.read_csv(filename, delimiter='\t')
                    ax.plot(data['colour'], data['distribution'], color=colour, linewidth=1.5,
                           label=f'{param_values[var]:.3g}')
        
        # Customize subplot
        if plot_type == 'uvlf':
            ax.set_xlim(-25, -16)
            ax.set_ylim(1e-7, 1e-3)
        else:
            ax.set_xlim(*colour_limits)
        
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.set_title(f'p{param_num}: {param_info["ParamName"]}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)
        
        # Add legend to first subplot only
        if param_num == 1:
            ax.legend(fontsize=6, title='Values', title_fontsize=6)
        
        # Add LogFlag info
        ax.text(0.02, 0.98, f'LogFlag: {param_info["LogFlag"]}',
                transform=ax.transAxes, fontsize=6, va='top')
    
    # Add overall title
    if plot_type == 'uvlf':
        title = f'UVLF Parameter Variations (z = {redshift["redshift"]})'
    else:
        title = f'Colour Distribution Parameter Variations (z = {redshift["redshift"]})'
    
    fig.suptitle(title, fontsize=16, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Save plot with updated directory structure
    if plot_type == 'uvlf':
        output_dir = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band_or_colour_pairs),
                                redshift['label'])
        filename = f'UVLF_all_parameters.pdf'
    else:
        # For colour plots, use new directory naming
        band1, band2 = band_or_colour_pairs[0]
        colour_dir = get_colour_dir_name(band1, band2)
        output_dir = os.path.join(plots_dir, 'colours', band_type, colour_dir,
                                redshift['label'])

        file_name = f"All_colours_{colour_dir}_z{redshift['label']}.pdf"
        filename = os.path.join(output_dir, file_name)

    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, filename), bbox_inches='tight', dpi=300)
    plt.close()



In [ ]:
def process_parameter_grids():
    """Process both UVLF and colour parameter grids for all redshifts."""
    # Read parameter information
    params_df = pd.read_csv(param_info_file)
    
    # Process each redshift
    for snap, redshift in redshift_values.items():
        print(f"\nProcessing redshift {redshift['redshift']}")
        
        # Process each band type
        for band_type in ['intrinsic', 'attenuated']:
            # Process UVLFs
            for band in filters[band_type]:
                print(f"Creating UVLF parameter grid for {band_type} {band}")
                create_parameter_grid(
                    params_df=params_df,
                    redshift=redshift,
                    band_type=band_type,
                    band_or_colour_pairs=band,
                    plot_type='uvlf'
                )
            
            # Process colours
            colour_pairs = [("GALEX FUV", "GALEX NUV")]  # Add more pairs if needed
            print(f"Creating colour parameter grid for {band_type}")
            create_parameter_grid(
                params_df=params_df,
                redshift=redshift,
                band_type=band_type,
                band_or_colour_pairs=colour_pairs,
                plot_type='colour'
            )

if __name__ == "__main__":
    process_parameter_grids()